In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import math
from math import log2

In [3]:
# PART A

train_set = pd.read_csv('spotify_train.csv')
test_set = pd.read_csv('spotify_test.csv')
train_set

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,211682,True,0.732,0.635,1,-7.891,1,0.4100,0.49300,0.000007,0.3880,0.581,147.025,4,hip-hop
1,1,223613,False,0.409,0.570,6,-10.540,0,0.0711,0.68700,0.000000,0.1730,0.336,128.657,4,pop
2,56,243626,False,0.536,0.764,1,-5.174,0,0.0393,0.03020,0.000011,0.1040,0.294,147.585,4,rock
3,8,182413,True,0.843,0.789,7,-2.801,1,0.2470,0.00280,0.000024,0.0322,0.571,125.071,4,hip-hop
4,74,200120,False,0.535,0.765,1,-7.862,0,0.0444,0.05400,0.000000,0.0921,0.373,191.827,4,pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,206696,False,0.741,0.695,1,-5.341,1,0.0454,0.02100,0.000000,0.0922,0.861,134.032,4,pop
1996,2,550706,False,0.765,0.833,10,-5.162,0,0.0873,0.03760,0.268000,0.0824,0.837,126.422,4,rock
1997,81,331266,False,0.577,0.828,6,-8.691,0,0.0298,0.00213,0.011400,0.0581,0.604,123.257,4,rock
1998,48,199263,False,0.754,0.802,9,-6.424,1,0.0551,0.03490,0.000000,0.3350,0.583,122.026,4,rock


In [4]:
y_vals = train_set.track_genre

x_vals = train_set.loc[:, train_set.columns != "track_genre"]

In [5]:
# Different Attempts at changing values
def oneDigit(num):
    if num == 0:
        return 0
    elif num < 0:
        num = -num
    while num < 1:
        num = num * 10
    while num > 10:
        num = num / 10
    return round(num * 1, 0)

def oneDigitFactor(num):
    factor = 1
    if num == 0:
        return 1
    elif num < 0:
        num = -num
        factor = -factor
    while num < 1:
        num = num * 10
        factor = factor * 10
    while num > 10:
        num = num / 10
        factor = factor / 10
    return factor

# Takes a number and keeps the highest value that isn't 0 and changes every other other digit to 0
def roundNum(num):
    makeZero = False
    nl = [int(d) for d in f'{num:f}' if d != '.' and d != '-']
    fnl = []
    for i in range(len(nl)):
        if makeZero:
            fnl.append(0)
        else:
            if (nl[i] != 0):
                makeZero = True
                fnl.append(nl[i])
    fnl.reverse()
    ret = 0
    for j in range(len(fnl)):
        ret += (fnl[j] * math.pow(10, j))
    return float(ret/1000000)

# Setting All Values so it is 10 times faster to 
vals_changed = train_set.copy()
vals_changed['popularity'] = vals_changed['popularity'].apply(lambda x: roundNum(x))
vals_changed['duration_ms'] = vals_changed['duration_ms'].apply(lambda x: roundNum(x/100000))
vals_changed['danceability'] = vals_changed['danceability'].apply(lambda x: roundNum(x*10))
vals_changed['energy'] = vals_changed['energy'].apply(lambda x: roundNum(x*10))
vals_changed['loudness'] = vals_changed['loudness'].apply(lambda x: roundNum(x*-1))
vals_changed['speechiness'] = vals_changed['speechiness'].apply(lambda x: roundNum(x*100))
vals_changed['acousticness'] = vals_changed['acousticness'].apply(lambda x: roundNum(x*1000))
vals_changed['instrumentalness'] = vals_changed['instrumentalness'].apply(lambda x: roundNum(x*1000000))
vals_changed['liveness'] = vals_changed['liveness'].apply(lambda x: roundNum(x*100))
vals_changed['valence'] = vals_changed['valence'].apply(lambda x: roundNum(x*10))
vals_changed['tempo'] = vals_changed['tempo'].apply(lambda x: roundNum(x/10))

vals_changed

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0.0,2.0,True,7.0,6.0,1,7.0,1,40.0,400.0,7.0,30.0,5.0,10.0,4,hip-hop
1,1.0,2.0,False,4.0,5.0,6,10.0,0,7.0,600.0,0.0,10.0,3.0,10.0,4,pop
2,50.0,2.0,False,5.0,7.0,1,5.0,0,3.0,30.0,10.0,10.0,2.0,10.0,4,rock
3,8.0,1.0,True,8.0,7.0,7,2.0,1,20.0,2.0,20.0,3.0,5.0,10.0,4,hip-hop
4,70.0,2.0,False,5.0,7.0,1,7.0,0,4.0,50.0,0.0,9.0,3.0,10.0,4,pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,2.0,False,7.0,6.0,1,5.0,1,4.0,20.0,0.0,9.0,8.0,10.0,4,pop
1996,2.0,5.0,False,7.0,8.0,10,5.0,0,8.0,30.0,200000.0,8.0,8.0,10.0,4,rock
1997,80.0,3.0,False,5.0,8.0,6,8.0,0,2.0,2.0,10000.0,5.0,6.0,10.0,4,rock
1998,40.0,1.0,False,7.0,8.0,9,6.0,1,5.0,30.0,0.0,30.0,5.0,10.0,4,rock


In [6]:
for i in range(len(vals_changed.columns)):
    ret = []
    for k in (vals_changed.iloc[:, i]):
        ret.append(k)
    print(len(Counter(ret)))
    print(i, vals_changed.iloc[:, i].name, Counter(ret))

20
0 popularity Counter({0.0: 627, 60.0: 418, 70.0: 247, 80.0: 181, 1.0: 146, 50.0: 128, 2.0: 84, 3.0: 49, 4.0: 40, 40.0: 17, 90.0: 17, 5.0: 16, 7.0: 7, 6.0: 7, 30.0: 5, 8.0: 3, 10.0: 3, 9.0: 2, 20.0: 2, 100.0: 1})
11
1 duration_ms Counter({2.0: 1003, 1.0: 818, 3.0: 149, 4.0: 6, 5.0: 5, 0.9: 4, 0.6: 4, 0.8: 4, 0.7: 3, 0.4: 3, 6.0: 1})
2
2 explicit Counter({False: 1727, True: 273})
9
3 danceability Counter({7.0: 488, 6.0: 485, 5.0: 399, 8.0: 229, 4.0: 217, 3.0: 94, 9.0: 44, 2.0: 40, 1.0: 4})
10
4 energy Counter({6.0: 387, 7.0: 381, 5.0: 371, 8.0: 320, 4.0: 225, 9.0: 163, 3.0: 96, 2.0: 29, 1.0: 27, 0.9: 1})
12
5 key Counter({0: 221, 11: 214, 1: 191, 7: 183, 4: 182, 5: 171, 6: 168, 9: 167, 2: 159, 8: 142, 10: 133, 3: 69})
11
6 loudness Counter({4.0: 351, 5.0: 343, 6.0: 318, 7.0: 253, 10.0: 174, 3.0: 174, 9.0: 160, 8.0: 154, 2.0: 63, 1.0: 9, 20.0: 1})
2
7 mode Counter({1: 1147, 0: 853})
13
8 speechiness Counter({3.0: 446, 4.0: 299, 10.0: 266, 2.0: 241, 5.0: 192, 20.0: 123, 6.0: 120, 7.0: 8

In [7]:
def total(cnt):
    return sum(cnt.values())

#def accuracy(cnt):
#    return cnt.most_common(1)[0][1] / total(cnt)
    
def gini(cnt):
    tot = total(cnt)
    return 1 - sum([(v/tot)**2 for v in cnt.values()])

def entropy(cnt):
    tot = total(cnt)
    return sum([(-v/tot) * log2(v/tot) for v in cnt.values()])
    
def wavg(cnt1, cnt2, measure):
    tot1 = total(cnt1)
    tot2 = total(cnt2)
    tot = tot1 + tot2
    return (measure(cnt1) * tot1 + measure(cnt2) * tot2) / tot

def evaluate_split(df, class_col, split_col, feature_val, measure):
    df1, df2 = df[df[split_col] == feature_val], df[df[split_col] != feature_val]
    cnt1, cnt2 = Counter(df1[class_col.name]), Counter(df2[class_col.name])
    return wavg(cnt1, cnt2, measure)

def best_split_for_column(df, class_col, split_col, method):
    best_v = ''
    best_meas = float("inf")
    
    for v in set(df[split_col]):
        
        meas = evaluate_split(df, class_col, split_col, v, method)
        if  meas < best_meas:
            best_v = v
            best_meas = meas
    
    return best_v, best_meas


def best_split(df, class_col, method):
    best_col = 0
    best_v = ''
    best_meas = float("inf")
    
    for split_col in df.columns:
        if split_col != class_col.name:
            v, meas = best_split_for_column(df, class_col, split_col, method)
            if meas <= best_meas:
                best_v = v
                best_meas = meas
                best_col = split_col
                
    return best_col, best_v, best_meas


best_split(vals_changed, vals_changed.track_genre, gini)

('explicit', False, 0.628163371660187)

In [8]:
# # Some helper functions
def left(T):
    return T[5] if T is not None else None
    

def right(T):
    return T[6] if T is not None else None

def column(T):
    return T[0] if T is not None else None

def value(T):
    return T[1] if T is not None else None

def majority(T):
    return T[3] if T is not None else None


# The depth is the Maximum number of "steps" to any leaf
def depth(T):
    """ Return the maximum depth of the tree """
    if T is None:
        return -1
    else:
        return 1 + max(depth(left(T)), depth(right(T)))

In [9]:
def dtree(train, criterion, thisDepth=0, max_depth=None, min_instances=2, target_impurity=0.0):
    colName, val, impurity = best_split(train, train.iloc[:, -1], criterion)
    majorityClass = max(Counter(train.iloc[:, -1]))
    if (impurity == float("inf") or colName == 0 or val == '' or (len(train[train[colName] == val].iloc[:, -1])) >= len(train.iloc[:, -1])):
        return (colName, val, impurity, majorityClass, 1, None, None)
    leftDF = train[train[colName] != val]
    rightDF = train[train[colName] == val]
    
    
    if (max_depth == None):
        if (len(train.iloc[:, -1]) <= min_instances) or (impurity <= target_impurity):
            return (colName, val, impurity, majorityClass, 1, None, None)
        
        leftTree = dtree(leftDF, criterion, thisDepth=thisDepth+1, max_depth=max_depth, min_instances=min_instances, target_impurity=target_impurity)
        rightTree = dtree(rightDF, criterion, thisDepth=thisDepth+1, max_depth=max_depth, min_instances=min_instances, target_impurity=target_impurity)
        return (colName, val, impurity, majorityClass, 1 + max(depth(leftTree), depth(rightTree)), leftTree, rightTree)
    else:
        if (thisDepth >= max_depth) or (len(train.iloc[:, -1]) <= min_instances) or (impurity <= target_impurity):
            return (colName, val, impurity, majorityClass, 1, None, None)

        leftTree = dtree(leftDF, criterion, thisDepth=thisDepth+1, max_depth=max_depth, min_instances=min_instances, target_impurity=target_impurity)
        rightTree = dtree(rightDF, criterion, thisDepth=thisDepth+1, max_depth=max_depth, min_instances=min_instances, target_impurity=target_impurity)
        return (colName, val, impurity, majorityClass, 1 + max(depth(leftTree), depth(rightTree)), leftTree, rightTree)



In [10]:
noMaxDepth = dtree(vals_changed, gini)
noMaxDepth

('explicit',
 False,
 0.628163371660187,
 'rock',
 26,
 ('danceability',
  8.0,
  0.39702629062234984,
  'rock',
  18,
  ('danceability',
   9.0,
   0.4783798576902026,
   'rock',
   17,
   ('danceability',
    7.0,
    0.5298885608915129,
    'rock',
    16,
    ('energy',
     9.0,
     0.5962457421520818,
     'rock',
     15,
     ('speechiness',
      2.0,
      0.5618423664879888,
      'rock',
      14,
      ('popularity',
       80.0,
       0.5478534317746787,
       'rock',
       13,
       ('speechiness',
        5.0,
        0.510943396226415,
        'rock',
        12,
        ('acousticness',
         80.0,
         0.47659574468085103,
         'rock',
         11,
         ('valence',
          4.0,
          0.434685799109352,
          'rock',
          10,
          ('danceability',
           4.0,
           0.3989968080255359,
           'rock',
           9,
           ('valence',
            8.0,
            0.30303030303030304,
            'rock',
           

In [12]:
# PART B
def predictData(tree, data):
    if ((left(tree) == None) or (right(tree) == None)):
        return majority(tree)
    if data[column(tree)] == value(tree):
        return predictData(right(tree), data)
    else:
        return predictData(left(tree), data)


def predict(model, data):
    ret = []
    for i in range(len(data.index)):
        ret.append(predictData(model, data.iloc[i]))
    return ret

def accuracy(pred, true):
    count = 0
    for i in range(len(pred)):
        if (list(true)[i] == pred[i]):
            count += 1
    return(count / len(pred))


pred = predict(noMaxDepth, vals_changed)
print('Accuracy of no Max Depth without kFold: ',accuracy(pred, vals_changed.iloc[:, -1]))

def kFold(data, criterion, max_depth=None, min_instances=2, target_impurity=0.0, folds=10):
    totalCount = 0
    for f in range(folds):
        train_fold = data[data.index % folds != f]
        predTree = dtree(train_fold, criterion, max_depth=max_depth, min_instances=min_instances, target_impurity=target_impurity)
        
        valid_fold = data[data.index % folds == f]
        predictions = predict(predTree, valid_fold)
        totalCount += accuracy(predictions, valid_fold.iloc[:, -1])
    return totalCount / folds

print('Accuracy with kFold: ', kFold(vals_changed, gini, max_depth = 5))

Accuracy of no Max Depth without kFold:  0.8345
Accuracy with kFold:  0.37799999999999995


In [13]:
# Part C

# Finding best criterion
print("Gini: ", kFold(vals_changed, gini, max_depth = 5))
print("Entropy: ", kFold(vals_changed, entropy, max_depth = 5))
# Entropy is best

Gini:  0.37799999999999995
Entropy:  0.40449999999999997


In [ ]:
# Finding best depth
for i in range(15, 18):
    kFold(vals_changed, entropy, max_depth = i)
# Best depth at 17

In [ ]:
# Finding best depth
for i in range(4, 7):
    kFold(vals_changed, entropy, max_depth = 17, min_instances=i)
# best min instances at 5

In [18]:
print(kFold(vals_changed, entropy, max_depth = 17, min_instances=5))
officialDTree = dtree(vals_changed, entropy, max_depth = 17, min_instances=5)

0.6130000000000001


In [14]:
# Setting All Values so it is 10 times faster to 
test_vals_changed = test_set.copy()
test_vals_changed['popularity'] = test_vals_changed['popularity'].apply(lambda x: roundNum(x))
test_vals_changed['duration_ms'] = test_vals_changed['duration_ms'].apply(lambda x: roundNum(x/100000))
test_vals_changed['danceability'] = test_vals_changed['danceability'].apply(lambda x: roundNum(x*10))
test_vals_changed['energy'] = test_vals_changed['energy'].apply(lambda x: roundNum(x*10))
test_vals_changed['loudness'] = test_vals_changed['loudness'].apply(lambda x: roundNum(x*-1))
test_vals_changed['speechiness'] = test_vals_changed['speechiness'].apply(lambda x: roundNum(x*100))
test_vals_changed['acousticness'] = test_vals_changed['acousticness'].apply(lambda x: roundNum(x*1000))
test_vals_changed['instrumentalness'] = test_vals_changed['instrumentalness'].apply(lambda x: roundNum(x*1000000))
test_vals_changed['liveness'] = test_vals_changed['liveness'].apply(lambda x: roundNum(x*100))
test_vals_changed['valence'] = test_vals_changed['valence'].apply(lambda x: roundNum(x*10))
test_vals_changed['tempo'] = test_vals_changed['tempo'].apply(lambda x: roundNum(x/10))
test_vals_changed

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0.0,1.0,False,6.0,7.0,4,3.0,0,4.0,20.0,0.0,10.0,4.0,10.0,4,pop
1,60.0,0.8,False,6.0,3.0,0,7.0,0,3.0,900.0,30.0,10.0,4.0,10.0,4,pop
2,1.0,2.0,False,6.0,7.0,6,5.0,1,4.0,200.0,3000.0,30.0,9.0,10.0,4,rock
3,70.0,2.0,False,6.0,6.0,6,7.0,1,4.0,70.0,10.0,10.0,4.0,9.0,4,hip-hop
4,0.0,2.0,False,5.0,8.0,2,5.0,1,3.0,400.0,100.0,30.0,7.0,10.0,4,rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,80.0,2.0,False,6.0,4.0,10,7.0,1,8.0,300.0,1.0,6.0,2.0,8.0,4,pop
996,0.0,2.0,False,5.0,5.0,0,4.0,1,2.0,400.0,0.0,10.0,3.0,7.0,4,rock
997,60.0,1.0,False,3.0,8.0,8,4.0,1,3.0,10.0,0.0,30.0,5.0,10.0,4,rock
998,60.0,3.0,False,4.0,8.0,8,4.0,0,6.0,100.0,400.0,8.0,4.0,8.0,4,pop


In [17]:
officialPredictions = predict(officialDTree, test_vals_changed)
officialAccuracty = accuracy(officialPredictions, test_vals_changed.track_genre)
print('Final Accuracy: ', officialAccuracty)

Final Accuracy:  0.648
